___
# **TELEBOT EQUIPO VERDE OSCURO**
___

El **bot del clima** es una herramienta interactiva desarrollada con el propósito de proporcionar información detallada sobre el clima de diversas ciudades, teniendo en cuenta **medias mensuales de los últimos 5 años**. A través de un proceso conversacional, el bot solicita al usuario información clave como el nombre de la ciudad, el mes deseado y luego genera gráficos comparativos y resúmenes climáticos de la ciudad seleccionada. 

Además, genera un reporte en un pdf.

Por último, se preguntará al usuario si quiere saber más sobre la ciudad que visitara, y en caso de la respuesta positiva, se le mostrará en pantalla información de dicha ciudad extraida mediante la **API** de **Wikipedia**

Este bot está diseñado para ser fácil de usar y responde de manera precisa a las consultas sobre el clima.

**Resumen del Proceso**

1.- **Inicio** de la conversación con el comando /start, dando la bienvenida al usuario.

2.- Solicitación del nombre del usuario para **personalizar** la interacción.

3.- Petición de la **ciudad** sobre la cual se desea obtener información climática.

4.- Selección del **mes** en el cual el usuario desea obtener los datos climáticos.

5.- **Graficos** y Generación de un **pdf** con los **resultados con gráficos y resumen de estadísticos**.

6.- Pregunta por si se quiere saber más sobre la ciudad elegida, y se muestra en pantalla la **información** de la ciudad extraida de la **Wikipedia**.

7.- **Despedida** y opción de realizar una nueva consulta, cerrando la interacción de manera eficiente.


### **1.- Librerías y preparación del Telebot**
___

In [60]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os
import telebot
import re
from telebot import types
import pydub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from telebot.types import ReplyKeyboardMarkup, KeyboardButton
import warnings
import wikipediaapi
warnings.filterwarnings(action='ignore')

In [61]:
# Ignorar advertencias de librerías para evitar mensajes innecesarios
warnings.filterwarnings(action='ignore')

# Ruta del archivo CSV con los datos climáticos
csv_path = "datos.csv"

# Crear el bot de Telegram utilizando el token de la API proporcionado
API_TOKEN = "7761290773:AAH-2mKnpiRp8GfCAxLp1wZWV7CGiNnyr-8"
bot = telebot.TeleBot(API_TOKEN)

# Memoria del chatbot, es un diccionario que almacena información del usuario durante la conversación
chatbot_memory = {}

___
### **2.- Funciones**
___

In [62]:

# Función para procesar el archivo de audio (de formato OGG a texto)
def procesar_audio(audio_file):
    try:
        # Usamos pydub para convertir el archivo de audio OGG a formato WAV
        audio = pydub.AudioSegment.from_file(audio_file, format="ogg")
        wav_file = "voice.wav"
        audio.export(wav_file, format="wav")

        # Utilizamos la librería SpeechRecognition para convertir el archivo WAV en texto
        import speech_recognition as sr
        recognizer = sr.Recognizer()
        
        # Abrimos el archivo WAV y lo procesamos para obtener el texto
        with sr.AudioFile(wav_file) as source:
            audio_data = recognizer.record(source)
            texto = recognizer.recognize_google(audio_data, language="es-ES")
            return texto
    except Exception as e:
        # Si ocurre un error durante el proceso, lo capturamos y devolvemos un mensaje de error
        return f"Error al procesar el audio: {e}"


In [63]:
def generar_pdf(chat_id, ciudad, mes):
    # Definimos el nombre del archivo PDF
    pdf_filename = f"resumen_climatico_{ciudad}_{mes}.pdf"
    
    # Creamos un objeto canvas para el PDF
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    
    # Escribimos la información climática en el PDF
    df = pd.read_csv(csv_path)
    datos_ciudad = df[(df["Ciudad"] == ciudad) & (df["Mes"] == mes)].iloc[0]
    
    c.setFont("Helvetica", 12)
    c.drawString(30, 750, f"Resumen Climático de {ciudad} en {mes}:")
    c.drawString(30, 730, f"• Temperatura Media: {datos_ciudad['Temperatura_Media']}°C")
    c.drawString(30, 710, f"• Precipitación: {datos_ciudad['Precipitacion_mm']} mm")
    c.drawString(30, 690, f"• Horas de Sol: {datos_ciudad['Horas_Sol']} horas")
    c.drawString(30, 670, f"• Humedad Relativa: {datos_ciudad['Humedad_Relativa']}%")
    
    # Agregamos los gráficos generados al PDF
    c.drawImage("grafico_comparativo.png", 30, 200, width=550, height=400)  # Ajusta el tamaño de la imagen
    
    # Guardamos el archivo PDF
    c.save()

    # Enviamos el PDF al usuario
    with open(pdf_filename, "rb") as f:
        bot.send_document(chat_id, f)
    
    # Eliminamos el archivo PDF después de enviarlo
    os.remove(pdf_filename)

In [64]:
# Función para generar los gráficos comparativos (barras, líneas y un nuevo gráfico en lugar de la correlación)
def generar_grafico_comparativo(chat_id, ciudad, mes):
    df = pd.read_csv(csv_path)
    datos_ciudad = df[(df["Ciudad"] == ciudad) & (df["Mes"] == mes)]  # Filtramos los datos para la ciudad y mes seleccionados
    datos_mes = df[df["Mes"] == mes]  # Filtramos todos los datos para el mes seleccionado

    if datos_ciudad.empty:
        bot.send_message(chat_id, f"Lo siento, no tengo datos para {ciudad} en {mes}.")
        return

    # Creamos una figura grande para los gráficos
    plt.figure(figsize=(14, 10))

    # Gráfico de barras para la comparación de temperaturas
    plt.subplot(2, 2, 1)
    sns.barplot(data=datos_mes, x="Ciudad", y="Temperatura_Media", palette="Blues", ci=None)
    plt.title(f"Comparación de Temperaturas - {mes}")
    plt.xticks(rotation=45)
    
    # Resaltar la ciudad seleccionada en el gráfico de barras de temperaturas (en verde)
    for index, city in enumerate(datos_mes["Ciudad"].unique()):
        if city == ciudad:
            plt.gca().patches[index].set_color('green')  # Resaltamos en verde la ciudad seleccionada

    # Gráfico de barras para precipitaciones
    plt.subplot(2, 2, 2)
    sns.barplot(data=datos_mes, x="Ciudad", y="Precipitacion_mm", palette="Blues", ci=None)
    plt.title(f"Comparación de Precipitaciones - {mes}")
    plt.xticks(rotation=45)

    # Resaltar la ciudad seleccionada en el gráfico de barras de precipitaciones (en verde)
    for index, city in enumerate(datos_mes["Ciudad"].unique()):
        if city == ciudad:
            plt.gca().patches[index].set_color('green')  # Resaltamos en verde la ciudad seleccionada

    # Gráfico de evolución climática para la ciudad seleccionada
    datos_ciudad_completa = df[df["Ciudad"] == ciudad]
    plt.subplot(2, 2, 3)
    sns.lineplot(data=datos_ciudad_completa, x="Mes", y="Temperatura_Media", label="Temperatura Media", marker="o")
    sns.lineplot(data=datos_ciudad_completa, x="Mes", y="Horas_Sol", label="Horas de Sol", marker="o")
    plt.title(f"Evolución Climática en {ciudad}")
    plt.legend()
    plt.xticks(rotation=45)

    # Nuevo gráfico: Evolución de la Temperatura Media durante el mes
    plt.subplot(2, 2, 4)
    sns.lineplot(data=datos_ciudad_completa, x="Mes", y="Temperatura_Media", label="Temperatura Media", marker="o", color="orange")
    plt.title("Evolución de la Temperatura Media en el mes")
    plt.xticks(rotation=45)

    # Ajustamos el diseño de los gráficos y agregamos el título general
    plt.tight_layout()
    plt.suptitle(f"Análisis Climático de {ciudad} y Comparación en {mes}", y=1.02)
    plt.savefig("grafico_comparativo.png")  # Guardamos la imagen generada
    plt.close()  # Cerramos la figura

    # Enviamos el gráfico al usuario
    with open("grafico_comparativo.png", 'rb') as photo:
        bot.send_photo(chat_id, photo)
    generar_pdf(chat_id, ciudad, mes)



In [65]:
# Función para enviar el resumen de las métricas climáticas de la ciudad seleccionada
def resumen_climatico(chat_id, ciudad, mes):
    df = pd.read_csv(csv_path)
    datos_ciudad = df[(df["Ciudad"] == ciudad) & (df["Mes"] == mes)].iloc[0]  # Obtenemos los datos de la ciudad y mes

    # Creamos el mensaje con las métricas climáticas de la ciudad
    mensaje = (f"*Resumen climático de {ciudad} en {mes}:*\n\n"
               f"• **Temperatura Media**: {datos_ciudad['Temperatura_Media']}°C\n"
               f"• **Precipitación**: {datos_ciudad['Precipitacion_mm']} mm\n"
               f"• **Horas de Sol**: {datos_ciudad['Horas_Sol']} horas\n"
               f"• **Humedad Relativa**: {datos_ciudad['Humedad_Relativa']}%\n")

    # Enviamos el resumen al usuario en formato Markdown
    bot.send_message(chat_id, mensaje, parse_mode='Markdown')

In [66]:
# Definir un tamaño máximo de caracteres por mensaje 
MAX_MESSAGE_LENGTH = 2000 #(para que no se alargue mucho)

def enviar_mensajes_largos(chat_id, mensaje):
    """Envía el mensaje en partes si es más largo que el límite permitido,
    y trata de no cortar las oraciones por la mitad."""
    
    while len(mensaje) > MAX_MESSAGE_LENGTH:
        # Buscamos la última posición del punto dentro del límite
        limite = mensaje.rfind('.', 0, MAX_MESSAGE_LENGTH)  # Buscar punto dentro de los primeros 2000 caracteres

        if limite == -1:
            # Si no encontramos un punto, cortamos al final del límite
            limite = MAX_MESSAGE_LENGTH

        # Enviamos la parte del mensaje y la recortamos para el siguiente envío
        bot.send_message(chat_id, mensaje[:limite+1])
        mensaje = mensaje[limite+1:].strip()  # Recortamos la parte ya enviada

    # Enviamos el resto del mensaje (si hay)
    if mensaje:
        bot.send_message(chat_id, mensaje)

    # Ahora que hemos enviado el mensaje, verificamos si se ha alcanzado el límite máximo y no se envían más mensajes.
    # Aquí es donde puedes detener la ejecución si lo deseas.
    if len(mensaje) > MAX_MESSAGE_LENGTH:
        return  # Aquí terminamos sin enviar más partes


In [67]:
def obtener_info_extra(ciudad, chat_id):
    # Inicializamos Wikipedia con un user agent válido
    wiki_wiki = wikipediaapi.Wikipedia(
        language='es', 
        user_agent="MiBotClima/1.0 (miemail@dominio.com)"
    )
    
    # Buscamos la página de la ciudad
    pagina = wiki_wiki.page(ciudad)

    # Verificamos si la página existe
    if pagina.exists():
        # Extraemos el resumen y eliminamos las referencias
        resumen = pagina.summary
        resumen_limpio = re.sub(r'\[\d+\]', '', resumen)  # Elimina las referencias

        # Enviamos el resumen limpio de la ciudad (dividido si es necesario)
        enviar_mensajes_largos(chat_id, resumen_limpio)

        # Opcionalmente, puedes preguntar si el usuario desea más información
        bot.send_message(chat_id, "¿Te gustaría saber más sobre esta ciudad?", reply_markup=types.ReplyKeyboardMarkup(
            resize_keyboard=True, one_time_keyboard=True).add('Sí', 'No'))
        
    else:
        # Si no existe, informamos al usuario
        bot.send_message(chat_id, f"No pude encontrar información adicional sobre {ciudad} en Wikipedia.")

In [ ]:
# Función para procesar la respuesta sobre más información
def process_more_info_response(message):
    respuesta = message.text.lower()
    chat_id = message.chat.id

    if respuesta in ['sí', 'si']:
        ciudad = chatbot_memory[chat_id]['ciudad']  # Obtenemos la ciudad guardada en la memoria
        obtener_info_extra(ciudad, chat_id)  # Llamamos a la función para obtener más información sobre la ciudad
        # Volvemos a preguntar después de mostrar la información extra
        ask_for_more_info(message)
    elif respuesta == 'no':
        bot.send_message(chat_id, f"¡Gracias por usar el bot del clima! Si tienes otra consulta, escribe /start.", reply_markup=types.ReplyKeyboardRemove())
        chatbot_memory.pop(chat_id, None)  # Eliminamos la memoria del chat para limpiar los datos
    else:
        bot.send_message(chat_id, "Por favor, responde 'Sí' o 'No'.")
        ask_for_more_info(message)  # Repetimos la pregunta si la respuesta no es válida


# Función para preguntar si desea más información sobre la ciudad
def ask_for_more_info(message):
    chat_id = message.chat.id

    # Creamos los botones de Sí y No
    markup = ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    markup.add(KeyboardButton('Sí'), KeyboardButton('No'))

    # Enviamos el mensaje preguntando si desea más información
    bot.send_message(chat_id, "¿Te gustaría obtener más información sobre la ciudad? (Puedes escribir 'Sí' o 'No').", reply_markup=markup)

    # Registramos el siguiente paso para procesar la respuesta
    bot.register_next_step_handler_by_chat_id(chat_id, process_more_info_response)


___
### **3.- Puesta en marcha**
___

In [69]:
# Función que maneja el comando /start o /help
@bot.message_handler(commands=['start', 'help'])
def first_step(message):
    chat_id = message.chat.id
    chatbot_memory[chat_id] = {}  # Inicializamos la memoria del chat para el usuario

    ultimo_mensaje = bot.send_message(chat_id, "¡Hola! Bienvenido al bot del clima.\n\nPor favor, escribe tu nombre o envíalo como audio:")
    bot.register_next_step_handler(ultimo_mensaje, second_step)

# Función para procesar el nombre del usuario
def second_step(message):
    chat_id = message.chat.id

    if message.content_type == 'voice':
        file_info = bot.get_file(message.voice.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        with open("voice.ogg", 'wb') as f:
            f.write(downloaded_file)
        nombre = procesar_audio("voice.ogg")
    else:
        nombre = message.text

    chatbot_memory[chat_id]['nombre'] = nombre

    df = pd.read_csv(csv_path)
    ciudades = df["Ciudad"].unique()
    
    markup = ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for ciudad in ciudades:
        markup.add(KeyboardButton(ciudad))

    ultimo_mensaje = bot.send_message(chat_id, f"Encantado, {nombre}! Ahora selecciona o escribe una ciudad española que quieras visitar:", reply_markup=markup)
    bot.register_next_step_handler(ultimo_mensaje, third_step)

# Función para procesar la ciudad seleccionada
def third_step(message):
    chat_id = message.chat.id
    ciudad = message.text.capitalize()

    df = pd.read_csv(csv_path)
    ciudades = df["Ciudad"].unique()

    if ciudad in ciudades:
        chatbot_memory[chat_id]['ciudad'] = ciudad

        meses = df["Mes"].unique()
        markup = ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
        for mes in meses:
            markup.add(KeyboardButton(mes))

        ultimo_mensaje = bot.send_message(chat_id, f"Perfecto, {ciudad}. Ahora dime el mes que te interesa:", reply_markup=markup)
        bot.register_next_step_handler(ultimo_mensaje, fourth_step)
    else:
        bot.send_message(chat_id, "No he encontrado esa ciudad. Por favor, selecciona una de la lista.")
        ask_for_city(message)

# Función para pedir ciudad nuevamente
def ask_for_city(message):
    chat_id = message.chat.id
    df = pd.read_csv(csv_path)
    ciudades = df["Ciudad"].unique()

    markup = ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for ciudad in ciudades:
        markup.add(KeyboardButton(ciudad))

    ultimo_mensaje = bot.send_message(chat_id, "Por favor, selecciona o escribe una ciudad española que quieras visitar:", reply_markup=markup)
    bot.register_next_step_handler(ultimo_mensaje, third_step)

# Función para procesar el mes seleccionado
def fourth_step(message):
    chat_id = message.chat.id
    mes = message.text.capitalize()

    df = pd.read_csv(csv_path)
    meses = df["Mes"].unique()

    if mes in meses:
        chatbot_memory[chat_id]['mes'] = mes
        ciudad = chatbot_memory[chat_id]['ciudad']

        generar_grafico_comparativo(chat_id, ciudad, mes)
        resumen_climatico(chat_id, ciudad, mes)

        ask_for_more_info(message)
    else:
        bot.send_message(chat_id, "No he encontrado ese mes. Por favor, selecciona uno de la lista.")
        ask_for_month(message)

# Función para pedir mes nuevamente
def ask_for_month(message):
    chat_id = message.chat.id
    df = pd.read_csv(csv_path)
    meses = df["Mes"].unique()

    markup = ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for mes in meses:
        markup.add(KeyboardButton(mes))

    ultimo_mensaje = bot.send_message(chat_id, "Por favor, selecciona o escribe un mes válido de la lista:", reply_markup=markup)
    bot.register_next_step_handler(ultimo_mensaje, fourth_step)

# Iniciar el bot
bot.polling()